In [1]:
import pandas as pd

In [ ]:
import pandas as pd

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('linkedin_jobs.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 31 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   job_id                      123849 non-null  int64  
 1   company_name                122130 non-null  object 
 2   title                       123849 non-null  object 
 3   description                 123842 non-null  object 
 4   max_salary                  29793 non-null   float64
 5   pay_period                  36073 non-null   object 
 6   location                    123849 non-null  object 
 7   company_id                  122132 non-null  float64
 8   views                       122160 non-null  float64
 9   med_salary                  6280 non-null    float64
 10  min_salary                  29793 non-null   float64
 11  formatted_work_type         123849 non-null  object 
 12  applies                     23320 non-null   float64
 13  original_liste

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('linkedin_jobs.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 31 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   job_id                      123849 non-null  int64  
 1   company_name                122130 non-null  object 
 2   title                       123849 non-null  object 
 3   description                 123842 non-null  object 
 4   max_salary                  29793 non-null   float64
 5   pay_period                  36073 non-null   object 
 6   location                    123849 non-null  object 
 7   company_id                  122132 non-null  float64
 8   views                       122160 non-null  float64
 9   med_salary                  6280 non-null    float64
 10  min_salary                  29793 non-null   float64
 11  formatted_work_type         123849 non-null  object 
 12  applies                     23320 non-null   float64
 13  original_liste

In [ ]:
tfidf_columns = ['title', 'description']
df_tfidf = df[tfidf_columns].copy()

In [ ]:
print("Dataset Info:")
df_tfidf.info()

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   title        123849 non-null  object
 1   description  123842 non-null  object
dtypes: object(2)
memory usage: 1.9+ MB


In [ ]:
print("\nNull values in each column:")
print(df_tfidf.isnull().sum())


Null values in each column:
title          0
description    7
dtype: int64


In [ ]:
df_clean = df_tfidf.dropna(subset=['description'])

In [ ]:
print("\nSample data:")
df_clean.head()


Sample data:


,title,description
0,Marketing Coordinator,Job descriptionA leading real estate firm in N...
1,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ..."
2,Assitant Restaurant Manager,The National Exemplar is accepting application...
3,Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...
4,Service Technician,Looking for HVAC service tech with experience ...


In [ ]:
df_clean['combined_text'] = df_clean['title'] + ' | ' + df_clean['description']

C:\Users\akstrov\AppData\Local\Temp\ipykernel_16128\62390856.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['combined_text'] = df_clean['title'] + ' | ' + df_clean['description']


In [ ]:
print("Sample combined text:")
df_clean['combined_text'].head()

Sample combined text:


0    Marketing Coordinator | Job descriptionA leadi...
1    Mental Health Therapist/Counselor | At Aspen T...
2    Assitant Restaurant Manager | The National Exe...
3    Senior Elder Law / Trusts and Estates Associat...
4     Service Technician | Looking for HVAC service...
Name: combined_text, dtype: object

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

def preprocess_text(text):
    try:
        # Convert to lowercase
        text = str(text).lower()
        
        # Remove special characters and numbers
        text = re.sub(r'[^a-zA-Z\s]', ' ', text)
        
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text).strip()
        
        # Tokenize
        tokens = word_tokenize(text)
        
        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]
        
        # POS tagging
        pos_tags = nltk.pos_tag(tokens)
        
        # Lemmatize with POS
        lemmatizer = WordNetLemmatizer()
        lemmatized_tokens = []
        for token, tag in pos_tags:
            if tag.startswith('VB'):  # Verb
                lem_token = lemmatizer.lemmatize(token, pos='v')
            elif tag.startswith('NN'):  # Noun
                lem_token = lemmatizer.lemmatize(token, pos='n')
            elif tag.startswith('JJ'):  # Adjective
                lem_token = lemmatizer.lemmatize(token, pos='a')
            else:
                lem_token = lemmatizer.lemmatize(token)
            lemmatized_tokens.append(lem_token)
        
        return ' '.join(lemmatized_tokens)
        
    except Exception as e:
        print(f"Error processing text: {e}")
        return ''

# Test with the same text
test_text = "Marketing Coordinator | Job description: coordinating marketing campaigns, analyzing metrics, working with teams, managing projects"
processed = preprocess_text(test_text)

print("\nTest lemmatization:")
print("Original:", test_text)
print("Processed:", processed)


Test lemmatization:
Original: Marketing Coordinator | Job description: coordinating marketing campaigns, analyzing metrics, working with teams, managing projects
Processed: marketing coordinator job description coordinate marketing campaign analyze metric work team manage project


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
df_clean['processed_text'] = df_clean['combined_text'].apply(preprocess_text)

C:\Users\akstrov\AppData\Local\Temp\ipykernel_16128\1890915169.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['processed_text'] = df_clean['combined_text'].apply(preprocess_text)


In [ ]:
print("Original vs Processed Text:")
for orig, proc in zip(df_clean['combined_text'].head(2), df_clean['processed_text'].head(2)):
    print("\nOriginal:", orig[:200], "...")
    print("Processed:", proc[:200], "...")

Original vs Processed Text:

Original: Marketing Coordinator | Job descriptionA leading real estate firm in New Jersey is seeking an administrative Marketing Coordinator with some experience in graphic design. You will be working closely w ...
Processed: marketing coordinator job descriptiona lead real estate firm new jersey seek administrative marketing coordinator experience graphic design work closely fun kind ambitious member sale team dynamic exe ...

Original: Mental Health Therapist/Counselor | At Aspen Therapy and Wellness , we are committed to serving clients with best practices to help them with change, improvements and better quality of life. We believ ...
Processed: mental health therapist counselor aspen therapy wellness commit serve client best practice help change improvement better quality life believe provide secure supportive environment grow clinician lear ...


In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Create TFIDF vectorizer
tfidf = TfidfVectorizer(
    max_features=1000,  # Limit features to most frequent 1000 terms
    min_df=5,          # Ignore terms that appear in less than 5 documents
    max_df=0.95,       # Ignore terms that appear in more than 95% of documents
    ngram_range=(1, 2) # Include both unigrams and bigrams
)

# Fit and transform the preprocessed text
tfidf_matrix = tfidf.fit_transform(df_clean['processed_text'])

# Get feature names (vocabulary)
feature_names = tfidf.get_feature_names_out()

# Print some statistics
print(f"Shape of TFIDF matrix: {tfidf_matrix.shape}")
print(f"\nNumber of features: {len(feature_names)}")
print("\nSample features (first 20):")
print(feature_names[:20])

# Show sample vector
print("\nSample document vector (first document):")
sample_vector = tfidf_matrix[0].toarray()[0]
sample_terms = [(term, score) for term, score in zip(feature_names, sample_vector) if score > 0]
print("Non-zero terms and their TFIDF scores:")
for term, score in sorted(sample_terms, key=lambda x: x[1], reverse=True)[:10]:
    print(f"{term}: {score:.4f}")

Shape of TFIDF matrix: (123842, 1000)

Number of features: 1000

Sample features (first 20):
['ability' 'ability work' 'able' 'accept' 'access' 'accommodation'
 'accord' 'accordance' 'account' 'accountability' 'accounting' 'accredit'
 'accuracy' 'accurate' 'accurately' 'achieve' 'across' 'act' 'action'
 'active']

Sample document vector (first document):
Non-zero terms and their TFIDF scores:
marketing: 0.3849
design: 0.2246
event: 0.1816
brand: 0.1785
organize: 0.1768
coordinator: 0.1630
person: 0.1586
request: 0.1511
fun: 0.1494
market: 0.1435


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Sample user profiles
sample_users = [
    {
        'skills': ["Digital Marketing", "Brand Management", "Social Media", "Graphic Design"],
        'experience': 5.5,
        'education': ["bachelor, Marketing University", "master, Digital Marketing Institute"],
        'location': "New York, USA",
        'remote_ok': True
    },
    {
        'skills': ["Python", "FastAPI", "SQL", "Machine Learning"],
        'experience': 3.0,
        'education': ["bachelor, Computer Science University"],
        'location': "San Francisco, USA",
        'remote_ok': False
    }
]

# Sample job postings
sample_jobs = [
    {
        'title': "Senior Marketing Manager",
        'description': "Looking for an experienced marketing professional to lead our digital marketing initiatives.",
        'required_skills': ["Digital Marketing", "Brand Management", "Team Leadership"],
        'required_experience': 5.0,
        'required_education': "bachelor",
        'location': "New York, USA",
        'remote_ok': True
    },
    {
        'title': "Python Developer",
        'description': "Seeking a Python developer with experience in web frameworks and machine learning.",
        'required_skills': ["Python", "FastAPI", "SQL"],
        'required_experience': 2.0,
        'required_education': "bachelor",
        'location': "San Francisco, USA",
        'remote_ok': False
    }
]

In [ ]:
# Prepare text for vectorization
def prepare_user_text(user):
    text = ' '.join(user['skills'])
    text += ' ' + ' '.join([edu.split(',')[0] for edu in user['education']])  # Add education levels
    return text

def prepare_job_text(job):
    text = job['title'] + ' ' + job['description']
    text += ' ' + ' '.join(job['required_skills'])
    text += ' ' + job['required_education']
    return text

# Preprocess and vectorize
user_texts = [preprocess_text(prepare_user_text(u)) for u in sample_users]
job_texts = [preprocess_text(prepare_job_text(j)) for j in sample_jobs]

# Transform texts to TFIDF vectors
user_vectors = tfidf.transform(user_texts)
job_vectors = tfidf.transform(job_texts)

# Calculate similarity
similarities = cosine_similarity(user_vectors, job_vectors)

# Print results
for i, user in enumerate(sample_users):
    print(f"\nUser {i+1}")
    print(f"Skills: {user['skills']}")
    print(f"Experience: {user['experience']} years")
    print(f"Education: {user['education']}")
    print(f"Location: {user['location']}")
    print("Top job matches:")
    
    user_similarities = similarities[i]
    top_matches = sorted(enumerate(user_similarities), key=lambda x: x[1], reverse=True)
    
    for job_idx, score in top_matches:
        job = sample_jobs[job_idx]
        print(f"\n- {job['title']}")
        print(f"  Similarity Score: {score:.4f}")
        print(f"  Required Experience: {job['required_experience']} years")
        print(f"  Location: {job['location']}")
        print(f"  Remote: {job['remote_ok']}")


User 1
Skills: ['Digital Marketing', 'Brand Management', 'Social Media', 'Graphic Design']
Experience: 5.5 years
Education: ['bachelor, Marketing University', 'master, Digital Marketing Institute']
Location: New York, USA
Top job matches:

- Senior Marketing Manager
  Similarity Score: 0.5632
  Required Experience: 5.0 years
  Location: New York, USA
  Remote: True

- Python Developer
  Similarity Score: 0.0556
  Required Experience: 2.0 years
  Location: San Francisco, USA
  Remote: False

User 2
Skills: ['Python', 'FastAPI', 'SQL', 'Machine Learning']
Experience: 3.0 years
Education: ['bachelor, Computer Science University']
Location: San Francisco, USA
Top job matches:

- Python Developer
  Similarity Score: 0.4662
  Required Experience: 2.0 years
  Location: San Francisco, USA
  Remote: False

- Senior Marketing Manager
  Similarity Score: 0.0738
  Required Experience: 5.0 years
  Location: New York, USA
  Remote: True


In [ ]:
def calculate_match_score(user, job, similarity_score):
    # Base score from TFIDF similarity
    total_score = similarity_score * 0.6  # 60% weight for content similarity
    
    # Experience match (20% weight)
    exp_score = 0.0
    if user['experience'] >= job['required_experience']:
        exp_score = 0.2
    elif user['experience'] >= job['required_experience'] * 0.8:  # Allow 80% of required
        exp_score = 0.1
    total_score += exp_score
    
    # Location match (10% weight)
    location_score = 0.1 if (user['location'] == job['location'] or 
                           (job['remote_ok'] and user['remote_ok'])) else 0.0
    total_score += location_score
    
    # Education match (10% weight)
    education_levels = {'high school': 1, 'bachelor': 2, 'master': 3, 'phd': 4}
    user_highest_edu = max([education_levels.get(edu.split(',')[0].strip().lower(), 0) 
                          for edu in user['education']])
    required_edu = education_levels.get(job['required_education'].lower(), 0)
    education_score = 0.1 if user_highest_edu >= required_edu else 0.0
    total_score += education_score
    
    return total_score

# Print results with combined scoring
for i, user in enumerate(sample_users):
    print(f"\nUser {i+1}")
    print(f"Skills: {user['skills']}")
    print(f"Experience: {user['experience']} years")
    print(f"Education: {user['education']}")
    print(f"Location: {user['location']}")
    print("Top job matches:")
    
    # Calculate combined scores
    user_similarities = similarities[i]
    combined_scores = [(idx, calculate_match_score(user, job, sim_score)) 
                      for idx, (job, sim_score) in enumerate(zip(sample_jobs, user_similarities))]
    
    # Sort by combined score
    top_matches = sorted(combined_scores, key=lambda x: x[1], reverse=True)
    
    for job_idx, score in top_matches:
        job = sample_jobs[job_idx]
        print(f"\n- {job['title']}")
        print(f"  Overall Match Score: {score:.4f}")
        print(f"  TFIDF Similarity: {user_similarities[job_idx]:.4f}")
        print(f"  Required Experience: {job['required_experience']} years")
        print(f"  Required Education: {job['required_education']}")
        print(f"  Location: {job['location']}")
        print(f"  Remote: {job['remote_ok']}")


User 1
Skills: ['Digital Marketing', 'Brand Management', 'Social Media', 'Graphic Design']
Experience: 5.5 years
Education: ['bachelor, Marketing University', 'master, Digital Marketing Institute']
Location: New York, USA
Top job matches:

- Senior Marketing Manager
  Overall Match Score: 0.7379
  TFIDF Similarity: 0.5632
  Required Experience: 5.0 years
  Required Education: bachelor
  Location: New York, USA
  Remote: True

- Python Developer
  Overall Match Score: 0.3334
  TFIDF Similarity: 0.0556
  Required Experience: 2.0 years
  Required Education: bachelor
  Location: San Francisco, USA
  Remote: False

User 2
Skills: ['Python', 'FastAPI', 'SQL', 'Machine Learning']
Experience: 3.0 years
Education: ['bachelor, Computer Science University']
Location: San Francisco, USA
Top job matches:

- Python Developer
  Overall Match Score: 0.6797
  TFIDF Similarity: 0.4662
  Required Experience: 2.0 years
  Required Education: bachelor
  Location: San Francisco, USA
  Remote: False

- Senio

In [ ]:
tfidf_columns = ['title', 'description']
df_tfidf = df[tfidf_columns].copy()

In [ ]:
print("Dataset Info:")
df_tfidf.info()

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   title        123849 non-null  object
 1   description  123842 non-null  object
dtypes: object(2)
memory usage: 1.9+ MB


In [ ]:
print("\nNull values in each column:")
print(df_tfidf.isnull().sum())


Null values in each column:
title          0
description    7
dtype: int64


In [ ]:
df_clean = df_tfidf.dropna(subset=['description'])

In [ ]:
print("\nSample data:")
df_clean.head()


Sample data:


,title,description
0,Marketing Coordinator,Job descriptionA leading real estate firm in N...
1,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ..."
2,Assitant Restaurant Manager,The National Exemplar is accepting application...
3,Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...
4,Service Technician,Looking for HVAC service tech with experience ...


In [ ]:
df_clean['combined_text'] = df_clean['title'] + ' | ' + df_clean['description']

C:\Users\akstrov\AppData\Local\Temp\ipykernel_16128\62390856.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['combined_text'] = df_clean['title'] + ' | ' + df_clean['description']


In [ ]:
print("Sample combined text:")
df_clean['combined_text'].head()

Sample combined text:


0    Marketing Coordinator | Job descriptionA leadi...
1    Mental Health Therapist/Counselor | At Aspen T...
2    Assitant Restaurant Manager | The National Exe...
3    Senior Elder Law / Trusts and Estates Associat...
4     Service Technician | Looking for HVAC service...
Name: combined_text, dtype: object

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

def preprocess_text(text):
    try:
        # Convert to lowercase
        text = str(text).lower()
        
        # Remove special characters and numbers
        text = re.sub(r'[^a-zA-Z\s]', ' ', text)
        
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text).strip()
        
        # Tokenize
        tokens = word_tokenize(text)
        
        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]
        
        # POS tagging
        pos_tags = nltk.pos_tag(tokens)
        
        # Lemmatize with POS
        lemmatizer = WordNetLemmatizer()
        lemmatized_tokens = []
        for token, tag in pos_tags:
            if tag.startswith('VB'):  # Verb
                lem_token = lemmatizer.lemmatize(token, pos='v')
            elif tag.startswith('NN'):  # Noun
                lem_token = lemmatizer.lemmatize(token, pos='n')
            elif tag.startswith('JJ'):  # Adjective
                lem_token = lemmatizer.lemmatize(token, pos='a')
            else:
                lem_token = lemmatizer.lemmatize(token)
            lemmatized_tokens.append(lem_token)
        
        return ' '.join(lemmatized_tokens)
        
    except Exception as e:
        print(f"Error processing text: {e}")
        return ''

# Test with the same text
test_text = "Marketing Coordinator | Job description: coordinating marketing campaigns, analyzing metrics, working with teams, managing projects"
processed = preprocess_text(test_text)

print("\nTest lemmatization:")
print("Original:", test_text)
print("Processed:", processed)


Test lemmatization:
Original: Marketing Coordinator | Job description: coordinating marketing campaigns, analyzing metrics, working with teams, managing projects
Processed: marketing coordinator job description coordinate marketing campaign analyze metric work team manage project


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
df_clean['processed_text'] = df_clean['combined_text'].apply(preprocess_text)

C:\Users\akstrov\AppData\Local\Temp\ipykernel_16128\1890915169.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['processed_text'] = df_clean['combined_text'].apply(preprocess_text)


In [ ]:
print("Original vs Processed Text:")
for orig, proc in zip(df_clean['combined_text'].head(2), df_clean['processed_text'].head(2)):
    print("\nOriginal:", orig[:200], "...")
    print("Processed:", proc[:200], "...")

Original vs Processed Text:

Original: Marketing Coordinator | Job descriptionA leading real estate firm in New Jersey is seeking an administrative Marketing Coordinator with some experience in graphic design. You will be working closely w ...
Processed: marketing coordinator job descriptiona lead real estate firm new jersey seek administrative marketing coordinator experience graphic design work closely fun kind ambitious member sale team dynamic exe ...

Original: Mental Health Therapist/Counselor | At Aspen Therapy and Wellness , we are committed to serving clients with best practices to help them with change, improvements and better quality of life. We believ ...
Processed: mental health therapist counselor aspen therapy wellness commit serve client best practice help change improvement better quality life believe provide secure supportive environment grow clinician lear ...


In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Create TFIDF vectorizer
tfidf = TfidfVectorizer(
    max_features=1000,  # Limit features to most frequent 1000 terms
    min_df=5,          # Ignore terms that appear in less than 5 documents
    max_df=0.95,       # Ignore terms that appear in more than 95% of documents
    ngram_range=(1, 2) # Include both unigrams and bigrams
)

# Fit and transform the preprocessed text
tfidf_matrix = tfidf.fit_transform(df_clean['processed_text'])

# Get feature names (vocabulary)
feature_names = tfidf.get_feature_names_out()

# Print some statistics
print(f"Shape of TFIDF matrix: {tfidf_matrix.shape}")
print(f"\nNumber of features: {len(feature_names)}")
print("\nSample features (first 20):")
print(feature_names[:20])

# Show sample vector
print("\nSample document vector (first document):")
sample_vector = tfidf_matrix[0].toarray()[0]
sample_terms = [(term, score) for term, score in zip(feature_names, sample_vector) if score > 0]
print("Non-zero terms and their TFIDF scores:")
for term, score in sorted(sample_terms, key=lambda x: x[1], reverse=True)[:10]:
    print(f"{term}: {score:.4f}")

Shape of TFIDF matrix: (123842, 1000)

Number of features: 1000

Sample features (first 20):
['ability' 'ability work' 'able' 'accept' 'access' 'accommodation'
 'accord' 'accordance' 'account' 'accountability' 'accounting' 'accredit'
 'accuracy' 'accurate' 'accurately' 'achieve' 'across' 'act' 'action'
 'active']

Sample document vector (first document):
Non-zero terms and their TFIDF scores:
marketing: 0.3849
design: 0.2246
event: 0.1816
brand: 0.1785
organize: 0.1768
coordinator: 0.1630
person: 0.1586
request: 0.1511
fun: 0.1494
market: 0.1435


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Sample user profiles
sample_users = [
    {
        'skills': ["Digital Marketing", "Brand Management", "Social Media", "Graphic Design"],
        'experience': 5.5,
        'education': ["bachelor, Marketing University", "master, Digital Marketing Institute"],
        'location': "New York, USA",
        'remote_ok': True
    },
    {
        'skills': ["Python", "FastAPI", "SQL", "Machine Learning"],
        'experience': 3.0,
        'education': ["bachelor, Computer Science University"],
        'location': "San Francisco, USA",
        'remote_ok': False
    }
]

# Sample job postings
sample_jobs = [
    {
        'title': "Senior Marketing Manager",
        'description': "Looking for an experienced marketing professional to lead our digital marketing initiatives.",
        'required_skills': ["Digital Marketing", "Brand Management", "Team Leadership"],
        'required_experience': 5.0,
        'required_education': "bachelor",
        'location': "New York, USA",
        'remote_ok': True
    },
    {
        'title': "Python Developer",
        'description': "Seeking a Python developer with experience in web frameworks and machine learning.",
        'required_skills': ["Python", "FastAPI", "SQL"],
        'required_experience': 2.0,
        'required_education': "bachelor",
        'location': "San Francisco, USA",
        'remote_ok': False
    }
]

In [ ]:
# Prepare text for vectorization
def prepare_user_text(user):
    text = ' '.join(user['skills'])
    text += ' ' + ' '.join([edu.split(',')[0] for edu in user['education']])  # Add education levels
    return text

def prepare_job_text(job):
    text = job['title'] + ' ' + job['description']
    text += ' ' + ' '.join(job['required_skills'])
    text += ' ' + job['required_education']
    return text

# Preprocess and vectorize
user_texts = [preprocess_text(prepare_user_text(u)) for u in sample_users]
job_texts = [preprocess_text(prepare_job_text(j)) for j in sample_jobs]

# Transform texts to TFIDF vectors
user_vectors = tfidf.transform(user_texts)
job_vectors = tfidf.transform(job_texts)

# Calculate similarity
similarities = cosine_similarity(user_vectors, job_vectors)

# Print results
for i, user in enumerate(sample_users):
    print(f"\nUser {i+1}")
    print(f"Skills: {user['skills']}")
    print(f"Experience: {user['experience']} years")
    print(f"Education: {user['education']}")
    print(f"Location: {user['location']}")
    print("Top job matches:")
    
    user_similarities = similarities[i]
    top_matches = sorted(enumerate(user_similarities), key=lambda x: x[1], reverse=True)
    
    for job_idx, score in top_matches:
        job = sample_jobs[job_idx]
        print(f"\n- {job['title']}")
        print(f"  Similarity Score: {score:.4f}")
        print(f"  Required Experience: {job['required_experience']} years")
        print(f"  Location: {job['location']}")
        print(f"  Remote: {job['remote_ok']}")


User 1
Skills: ['Digital Marketing', 'Brand Management', 'Social Media', 'Graphic Design']
Experience: 5.5 years
Education: ['bachelor, Marketing University', 'master, Digital Marketing Institute']
Location: New York, USA
Top job matches:

- Senior Marketing Manager
  Similarity Score: 0.5632
  Required Experience: 5.0 years
  Location: New York, USA
  Remote: True

- Python Developer
  Similarity Score: 0.0556
  Required Experience: 2.0 years
  Location: San Francisco, USA
  Remote: False

User 2
Skills: ['Python', 'FastAPI', 'SQL', 'Machine Learning']
Experience: 3.0 years
Education: ['bachelor, Computer Science University']
Location: San Francisco, USA
Top job matches:

- Python Developer
  Similarity Score: 0.4662
  Required Experience: 2.0 years
  Location: San Francisco, USA
  Remote: False

- Senior Marketing Manager
  Similarity Score: 0.0738
  Required Experience: 5.0 years
  Location: New York, USA
  Remote: True


In [ ]:
def calculate_match_score(user, job, similarity_score):
    # Base score from TFIDF similarity
    total_score = similarity_score * 0.6  # 60% weight for content similarity
    
    # Experience match (20% weight)
    exp_score = 0.0
    if user['experience'] >= job['required_experience']:
        exp_score = 0.2
    elif user['experience'] >= job['required_experience'] * 0.8:  # Allow 80% of required
        exp_score = 0.1
    total_score += exp_score
    
    # Location match (10% weight)
    location_score = 0.1 if (user['location'] == job['location'] or 
                           (job['remote_ok'] and user['remote_ok'])) else 0.0
    total_score += location_score
    
    # Education match (10% weight)
    education_levels = {'high school': 1, 'bachelor': 2, 'master': 3, 'phd': 4}
    user_highest_edu = max([education_levels.get(edu.split(',')[0].strip().lower(), 0) 
                          for edu in user['education']])
    required_edu = education_levels.get(job['required_education'].lower(), 0)
    education_score = 0.1 if user_highest_edu >= required_edu else 0.0
    total_score += education_score
    
    return total_score

# Print results with combined scoring
for i, user in enumerate(sample_users):
    print(f"\nUser {i+1}")
    print(f"Skills: {user['skills']}")
    print(f"Experience: {user['experience']} years")
    print(f"Education: {user['education']}")
    print(f"Location: {user['location']}")
    print("Top job matches:")
    
    # Calculate combined scores
    user_similarities = similarities[i]
    combined_scores = [(idx, calculate_match_score(user, job, sim_score)) 
                      for idx, (job, sim_score) in enumerate(zip(sample_jobs, user_similarities))]
    
    # Sort by combined score
    top_matches = sorted(combined_scores, key=lambda x: x[1], reverse=True)
    
    for job_idx, score in top_matches:
        job = sample_jobs[job_idx]
        print(f"\n- {job['title']}")
        print(f"  Overall Match Score: {score:.4f}")
        print(f"  TFIDF Similarity: {user_similarities[job_idx]:.4f}")
        print(f"  Required Experience: {job['required_experience']} years")
        print(f"  Required Education: {job['required_education']}")
        print(f"  Location: {job['location']}")
        print(f"  Remote: {job['remote_ok']}")


User 1
Skills: ['Digital Marketing', 'Brand Management', 'Social Media', 'Graphic Design']
Experience: 5.5 years
Education: ['bachelor, Marketing University', 'master, Digital Marketing Institute']
Location: New York, USA
Top job matches:

- Senior Marketing Manager
  Overall Match Score: 0.7379
  TFIDF Similarity: 0.5632
  Required Experience: 5.0 years
  Required Education: bachelor
  Location: New York, USA
  Remote: True

- Python Developer
  Overall Match Score: 0.3334
  TFIDF Similarity: 0.0556
  Required Experience: 2.0 years
  Required Education: bachelor
  Location: San Francisco, USA
  Remote: False

User 2
Skills: ['Python', 'FastAPI', 'SQL', 'Machine Learning']
Experience: 3.0 years
Education: ['bachelor, Computer Science University']
Location: San Francisco, USA
Top job matches:

- Python Developer
  Overall Match Score: 0.6797
  TFIDF Similarity: 0.4662
  Required Experience: 2.0 years
  Required Education: bachelor
  Location: San Francisco, USA
  Remote: False

- Senio

In [ ]:
df = pd.read_csv('linkedin_jobs.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 31 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   job_id                      123849 non-null  int64  
 1   company_name                122130 non-null  object 
 2   title                       123849 non-null  object 
 3   description                 123842 non-null  object 
 4   max_salary                  29793 non-null   float64
 5   pay_period                  36073 non-null   object 
 6   location                    123849 non-null  object 
 7   company_id                  122132 non-null  float64
 8   views                       122160 non-null  float64
 9   med_salary                  6280 non-null    float64
 10  min_salary                  29793 non-null   float64
 11  formatted_work_type         123849 non-null  object 
 12  applies                     23320 non-null   float64
 13  original_liste

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('linkedin_jobs.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 31 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   job_id                      123849 non-null  int64  
 1   company_name                122130 non-null  object 
 2   title                       123849 non-null  object 
 3   description                 123842 non-null  object 
 4   max_salary                  29793 non-null   float64
 5   pay_period                  36073 non-null   object 
 6   location                    123849 non-null  object 
 7   company_id                  122132 non-null  float64
 8   views                       122160 non-null  float64
 9   med_salary                  6280 non-null    float64
 10  min_salary                  29793 non-null   float64
 11  formatted_work_type         123849 non-null  object 
 12  applies                     23320 non-null   float64
 13  original_liste

In [ ]:
tfidf_columns = ['title', 'description']
df_tfidf = df[tfidf_columns].copy()

In [ ]:
print("Dataset Info:")
df_tfidf.info()

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   title        123849 non-null  object
 1   description  123842 non-null  object
dtypes: object(2)
memory usage: 1.9+ MB


In [ ]:
print("\nNull values in each column:")
print(df_tfidf.isnull().sum())


Null values in each column:
title          0
description    7
dtype: int64


In [ ]:
df_clean = df_tfidf.dropna(subset=['description'])

In [ ]:
print("\nSample data:")
df_clean.head()


Sample data:


,title,description
0,Marketing Coordinator,Job descriptionA leading real estate firm in N...
1,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ..."
2,Assitant Restaurant Manager,The National Exemplar is accepting application...
3,Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...
4,Service Technician,Looking for HVAC service tech with experience ...


In [ ]:
df_clean['combined_text'] = df_clean['title'] + ' | ' + df_clean['description']

C:\Users\akstrov\AppData\Local\Temp\ipykernel_16128\62390856.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['combined_text'] = df_clean['title'] + ' | ' + df_clean['description']


In [ ]:
print("Sample combined text:")
df_clean['combined_text'].head()

Sample combined text:


0    Marketing Coordinator | Job descriptionA leadi...
1    Mental Health Therapist/Counselor | At Aspen T...
2    Assitant Restaurant Manager | The National Exe...
3    Senior Elder Law / Trusts and Estates Associat...
4     Service Technician | Looking for HVAC service...
Name: combined_text, dtype: object

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

def preprocess_text(text):
    try:
        # Convert to lowercase
        text = str(text).lower()
        
        # Remove special characters and numbers
        text = re.sub(r'[^a-zA-Z\s]', ' ', text)
        
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text).strip()
        
        # Tokenize
        tokens = word_tokenize(text)
        
        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]
        
        # POS tagging
        pos_tags = nltk.pos_tag(tokens)
        
        # Lemmatize with POS
        lemmatizer = WordNetLemmatizer()
        lemmatized_tokens = []
        for token, tag in pos_tags:
            if tag.startswith('VB'):  # Verb
                lem_token = lemmatizer.lemmatize(token, pos='v')
            elif tag.startswith('NN'):  # Noun
                lem_token = lemmatizer.lemmatize(token, pos='n')
            elif tag.startswith('JJ'):  # Adjective
                lem_token = lemmatizer.lemmatize(token, pos='a')
            else:
                lem_token = lemmatizer.lemmatize(token)
            lemmatized_tokens.append(lem_token)
        
        return ' '.join(lemmatized_tokens)
        
    except Exception as e:
        print(f"Error processing text: {e}")
        return ''

# Test with the same text
test_text = "Marketing Coordinator | Job description: coordinating marketing campaigns, analyzing metrics, working with teams, managing projects"
processed = preprocess_text(test_text)

print("\nTest lemmatization:")
print("Original:", test_text)
print("Processed:", processed)


Test lemmatization:
Original: Marketing Coordinator | Job description: coordinating marketing campaigns, analyzing metrics, working with teams, managing projects
Processed: marketing coordinator job description coordinate marketing campaign analyze metric work team manage project


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
df_clean['processed_text'] = df_clean['combined_text'].apply(preprocess_text)

C:\Users\akstrov\AppData\Local\Temp\ipykernel_16128\1890915169.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['processed_text'] = df_clean['combined_text'].apply(preprocess_text)


In [ ]:
print("Original vs Processed Text:")
for orig, proc in zip(df_clean['combined_text'].head(2), df_clean['processed_text'].head(2)):
    print("\nOriginal:", orig[:200], "...")
    print("Processed:", proc[:200], "...")

Original vs Processed Text:

Original: Marketing Coordinator | Job descriptionA leading real estate firm in New Jersey is seeking an administrative Marketing Coordinator with some experience in graphic design. You will be working closely w ...
Processed: marketing coordinator job descriptiona lead real estate firm new jersey seek administrative marketing coordinator experience graphic design work closely fun kind ambitious member sale team dynamic exe ...

Original: Mental Health Therapist/Counselor | At Aspen Therapy and Wellness , we are committed to serving clients with best practices to help them with change, improvements and better quality of life. We believ ...
Processed: mental health therapist counselor aspen therapy wellness commit serve client best practice help change improvement better quality life believe provide secure supportive environment grow clinician lear ...


In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Create TFIDF vectorizer
tfidf = TfidfVectorizer(
    max_features=1000,  # Limit features to most frequent 1000 terms
    min_df=5,          # Ignore terms that appear in less than 5 documents
    max_df=0.95,       # Ignore terms that appear in more than 95% of documents
    ngram_range=(1, 2) # Include both unigrams and bigrams
)

# Fit and transform the preprocessed text
tfidf_matrix = tfidf.fit_transform(df_clean['processed_text'])

# Get feature names (vocabulary)
feature_names = tfidf.get_feature_names_out()

# Print some statistics
print(f"Shape of TFIDF matrix: {tfidf_matrix.shape}")
print(f"\nNumber of features: {len(feature_names)}")
print("\nSample features (first 20):")
print(feature_names[:20])

# Show sample vector
print("\nSample document vector (first document):")
sample_vector = tfidf_matrix[0].toarray()[0]
sample_terms = [(term, score) for term, score in zip(feature_names, sample_vector) if score > 0]
print("Non-zero terms and their TFIDF scores:")
for term, score in sorted(sample_terms, key=lambda x: x[1], reverse=True)[:10]:
    print(f"{term}: {score:.4f}")

Shape of TFIDF matrix: (123842, 1000)

Number of features: 1000

Sample features (first 20):
['ability' 'ability work' 'able' 'accept' 'access' 'accommodation'
 'accord' 'accordance' 'account' 'accountability' 'accounting' 'accredit'
 'accuracy' 'accurate' 'accurately' 'achieve' 'across' 'act' 'action'
 'active']

Sample document vector (first document):
Non-zero terms and their TFIDF scores:
marketing: 0.3849
design: 0.2246
event: 0.1816
brand: 0.1785
organize: 0.1768
coordinator: 0.1630
person: 0.1586
request: 0.1511
fun: 0.1494
market: 0.1435


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Sample user profiles
sample_users = [
    {
        'skills': ["Digital Marketing", "Brand Management", "Social Media", "Graphic Design"],
        'experience': 5.5,
        'education': ["bachelor, Marketing University", "master, Digital Marketing Institute"],
        'location': "New York, USA",
        'remote_ok': True
    },
    {
        'skills': ["Python", "FastAPI", "SQL", "Machine Learning"],
        'experience': 3.0,
        'education': ["bachelor, Computer Science University"],
        'location': "San Francisco, USA",
        'remote_ok': False
    }
]

# Sample job postings
sample_jobs = [
    {
        'title': "Senior Marketing Manager",
        'description': "Looking for an experienced marketing professional to lead our digital marketing initiatives.",
        'required_skills': ["Digital Marketing", "Brand Management", "Team Leadership"],
        'required_experience': 5.0,
        'required_education': "bachelor",
        'location': "New York, USA",
        'remote_ok': True
    },
    {
        'title': "Python Developer",
        'description': "Seeking a Python developer with experience in web frameworks and machine learning.",
        'required_skills': ["Python", "FastAPI", "SQL"],
        'required_experience': 2.0,
        'required_education': "bachelor",
        'location': "San Francisco, USA",
        'remote_ok': False
    }
]

In [ ]:
# Prepare text for vectorization
def prepare_user_text(user):
    text = ' '.join(user['skills'])
    text += ' ' + ' '.join([edu.split(',')[0] for edu in user['education']])  # Add education levels
    return text

def prepare_job_text(job):
    text = job['title'] + ' ' + job['description']
    text += ' ' + ' '.join(job['required_skills'])
    text += ' ' + job['required_education']
    return text

# Preprocess and vectorize
user_texts = [preprocess_text(prepare_user_text(u)) for u in sample_users]
job_texts = [preprocess_text(prepare_job_text(j)) for j in sample_jobs]

# Transform texts to TFIDF vectors
user_vectors = tfidf.transform(user_texts)
job_vectors = tfidf.transform(job_texts)

# Calculate similarity
similarities = cosine_similarity(user_vectors, job_vectors)

# Print results
for i, user in enumerate(sample_users):
    print(f"\nUser {i+1}")
    print(f"Skills: {user['skills']}")
    print(f"Experience: {user['experience']} years")
    print(f"Education: {user['education']}")
    print(f"Location: {user['location']}")
    print("Top job matches:")
    
    user_similarities = similarities[i]
    top_matches = sorted(enumerate(user_similarities), key=lambda x: x[1], reverse=True)
    
    for job_idx, score in top_matches:
        job = sample_jobs[job_idx]
        print(f"\n- {job['title']}")
        print(f"  Similarity Score: {score:.4f}")
        print(f"  Required Experience: {job['required_experience']} years")
        print(f"  Location: {job['location']}")
        print(f"  Remote: {job['remote_ok']}")


User 1
Skills: ['Digital Marketing', 'Brand Management', 'Social Media', 'Graphic Design']
Experience: 5.5 years
Education: ['bachelor, Marketing University', 'master, Digital Marketing Institute']
Location: New York, USA
Top job matches:

- Senior Marketing Manager
  Similarity Score: 0.5632
  Required Experience: 5.0 years
  Location: New York, USA
  Remote: True

- Python Developer
  Similarity Score: 0.0556
  Required Experience: 2.0 years
  Location: San Francisco, USA
  Remote: False

User 2
Skills: ['Python', 'FastAPI', 'SQL', 'Machine Learning']
Experience: 3.0 years
Education: ['bachelor, Computer Science University']
Location: San Francisco, USA
Top job matches:

- Python Developer
  Similarity Score: 0.4662
  Required Experience: 2.0 years
  Location: San Francisco, USA
  Remote: False

- Senior Marketing Manager
  Similarity Score: 0.0738
  Required Experience: 5.0 years
  Location: New York, USA
  Remote: True


In [ ]:
def calculate_match_score(user, job, similarity_score):
    # Base score from TFIDF similarity
    total_score = similarity_score * 0.6  # 60% weight for content similarity
    
    # Experience match (20% weight)
    exp_score = 0.0
    if user['experience'] >= job['required_experience']:
        exp_score = 0.2
    elif user['experience'] >= job['required_experience'] * 0.8:  # Allow 80% of required
        exp_score = 0.1
    total_score += exp_score
    
    # Location match (10% weight)
    location_score = 0.1 if (user['location'] == job['location'] or 
                           (job['remote_ok'] and user['remote_ok'])) else 0.0
    total_score += location_score
    
    # Education match (10% weight)
    education_levels = {'high school': 1, 'bachelor': 2, 'master': 3, 'phd': 4}
    user_highest_edu = max([education_levels.get(edu.split(',')[0].strip().lower(), 0) 
                          for edu in user['education']])
    required_edu = education_levels.get(job['required_education'].lower(), 0)
    education_score = 0.1 if user_highest_edu >= required_edu else 0.0
    total_score += education_score
    
    return total_score

# Print results with combined scoring
for i, user in enumerate(sample_users):
    print(f"\nUser {i+1}")
    print(f"Skills: {user['skills']}")
    print(f"Experience: {user['experience']} years")
    print(f"Education: {user['education']}")
    print(f"Location: {user['location']}")
    print("Top job matches:")
    
    # Calculate combined scores
    user_similarities = similarities[i]
    combined_scores = [(idx, calculate_match_score(user, job, sim_score)) 
                      for idx, (job, sim_score) in enumerate(zip(sample_jobs, user_similarities))]
    
    # Sort by combined score
    top_matches = sorted(combined_scores, key=lambda x: x[1], reverse=True)
    
    for job_idx, score in top_matches:
        job = sample_jobs[job_idx]
        print(f"\n- {job['title']}")
        print(f"  Overall Match Score: {score:.4f}")
        print(f"  TFIDF Similarity: {user_similarities[job_idx]:.4f}")
        print(f"  Required Experience: {job['required_experience']} years")
        print(f"  Required Education: {job['required_education']}")
        print(f"  Location: {job['location']}")
        print(f"  Remote: {job['remote_ok']}")


User 1
Skills: ['Digital Marketing', 'Brand Management', 'Social Media', 'Graphic Design']
Experience: 5.5 years
Education: ['bachelor, Marketing University', 'master, Digital Marketing Institute']
Location: New York, USA
Top job matches:

- Senior Marketing Manager
  Overall Match Score: 0.7379
  TFIDF Similarity: 0.5632
  Required Experience: 5.0 years
  Required Education: bachelor
  Location: New York, USA
  Remote: True

- Python Developer
  Overall Match Score: 0.3334
  TFIDF Similarity: 0.0556
  Required Experience: 2.0 years
  Required Education: bachelor
  Location: San Francisco, USA
  Remote: False

User 2
Skills: ['Python', 'FastAPI', 'SQL', 'Machine Learning']
Experience: 3.0 years
Education: ['bachelor, Computer Science University']
Location: San Francisco, USA
Top job matches:

- Python Developer
  Overall Match Score: 0.6797
  TFIDF Similarity: 0.4662
  Required Experience: 2.0 years
  Required Education: bachelor
  Location: San Francisco, USA
  Remote: False

- Senio

In [ ]:
tfidf_columns = ['title', 'description']
df_tfidf = df[tfidf_columns].copy()

In [ ]:
print("Dataset Info:")
df_tfidf.info()

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   title        123849 non-null  object
 1   description  123842 non-null  object
dtypes: object(2)
memory usage: 1.9+ MB


In [ ]:
print("\nNull values in each column:")
print(df_tfidf.isnull().sum())


Null values in each column:
title          0
description    7
dtype: int64


In [ ]:
df_clean = df_tfidf.dropna(subset=['description'])

In [ ]:
print("\nSample data:")
df_clean.head()


Sample data:


,title,description
0,Marketing Coordinator,Job descriptionA leading real estate firm in N...
1,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ..."
2,Assitant Restaurant Manager,The National Exemplar is accepting application...
3,Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...
4,Service Technician,Looking for HVAC service tech with experience ...


In [ ]:
df_clean['combined_text'] = df_clean['title'] + ' | ' + df_clean['description']

C:\Users\akstrov\AppData\Local\Temp\ipykernel_16128\62390856.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['combined_text'] = df_clean['title'] + ' | ' + df_clean['description']


In [ ]:
print("Sample combined text:")
df_clean['combined_text'].head()

Sample combined text:


0    Marketing Coordinator | Job descriptionA leadi...
1    Mental Health Therapist/Counselor | At Aspen T...
2    Assitant Restaurant Manager | The National Exe...
3    Senior Elder Law / Trusts and Estates Associat...
4     Service Technician | Looking for HVAC service...
Name: combined_text, dtype: object

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

def preprocess_text(text):
    try:
        # Convert to lowercase
        text = str(text).lower()
        
        # Remove special characters and numbers
        text = re.sub(r'[^a-zA-Z\s]', ' ', text)
        
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text).strip()
        
        # Tokenize
        tokens = word_tokenize(text)
        
        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]
        
        # POS tagging
        pos_tags = nltk.pos_tag(tokens)
        
        # Lemmatize with POS
        lemmatizer = WordNetLemmatizer()
        lemmatized_tokens = []
        for token, tag in pos_tags:
            if tag.startswith('VB'):  # Verb
                lem_token = lemmatizer.lemmatize(token, pos='v')
            elif tag.startswith('NN'):  # Noun
                lem_token = lemmatizer.lemmatize(token, pos='n')
            elif tag.startswith('JJ'):  # Adjective
                lem_token = lemmatizer.lemmatize(token, pos='a')
            else:
                lem_token = lemmatizer.lemmatize(token)
            lemmatized_tokens.append(lem_token)
        
        return ' '.join(lemmatized_tokens)
        
    except Exception as e:
        print(f"Error processing text: {e}")
        return ''

# Test with the same text
test_text = "Marketing Coordinator | Job description: coordinating marketing campaigns, analyzing metrics, working with teams, managing projects"
processed = preprocess_text(test_text)

print("\nTest lemmatization:")
print("Original:", test_text)
print("Processed:", processed)


Test lemmatization:
Original: Marketing Coordinator | Job description: coordinating marketing campaigns, analyzing metrics, working with teams, managing projects
Processed: marketing coordinator job description coordinate marketing campaign analyze metric work team manage project


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
df_clean['processed_text'] = df_clean['combined_text'].apply(preprocess_text)

C:\Users\akstrov\AppData\Local\Temp\ipykernel_16128\1890915169.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['processed_text'] = df_clean['combined_text'].apply(preprocess_text)


In [ ]:
print("Original vs Processed Text:")
for orig, proc in zip(df_clean['combined_text'].head(2), df_clean['processed_text'].head(2)):
    print("\nOriginal:", orig[:200], "...")
    print("Processed:", proc[:200], "...")

Original vs Processed Text:

Original: Marketing Coordinator | Job descriptionA leading real estate firm in New Jersey is seeking an administrative Marketing Coordinator with some experience in graphic design. You will be working closely w ...
Processed: marketing coordinator job descriptiona lead real estate firm new jersey seek administrative marketing coordinator experience graphic design work closely fun kind ambitious member sale team dynamic exe ...

Original: Mental Health Therapist/Counselor | At Aspen Therapy and Wellness , we are committed to serving clients with best practices to help them with change, improvements and better quality of life. We believ ...
Processed: mental health therapist counselor aspen therapy wellness commit serve client best practice help change improvement better quality life believe provide secure supportive environment grow clinician lear ...


In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Create TFIDF vectorizer
tfidf = TfidfVectorizer(
    max_features=1000,  # Limit features to most frequent 1000 terms
    min_df=5,          # Ignore terms that appear in less than 5 documents
    max_df=0.95,       # Ignore terms that appear in more than 95% of documents
    ngram_range=(1, 2) # Include both unigrams and bigrams
)

# Fit and transform the preprocessed text
tfidf_matrix = tfidf.fit_transform(df_clean['processed_text'])

# Get feature names (vocabulary)
feature_names = tfidf.get_feature_names_out()

# Print some statistics
print(f"Shape of TFIDF matrix: {tfidf_matrix.shape}")
print(f"\nNumber of features: {len(feature_names)}")
print("\nSample features (first 20):")
print(feature_names[:20])

# Show sample vector
print("\nSample document vector (first document):")
sample_vector = tfidf_matrix[0].toarray()[0]
sample_terms = [(term, score) for term, score in zip(feature_names, sample_vector) if score > 0]
print("Non-zero terms and their TFIDF scores:")
for term, score in sorted(sample_terms, key=lambda x: x[1], reverse=True)[:10]:
    print(f"{term}: {score:.4f}")

Shape of TFIDF matrix: (123842, 1000)

Number of features: 1000

Sample features (first 20):
['ability' 'ability work' 'able' 'accept' 'access' 'accommodation'
 'accord' 'accordance' 'account' 'accountability' 'accounting' 'accredit'
 'accuracy' 'accurate' 'accurately' 'achieve' 'across' 'act' 'action'
 'active']

Sample document vector (first document):
Non-zero terms and their TFIDF scores:
marketing: 0.3849
design: 0.2246
event: 0.1816
brand: 0.1785
organize: 0.1768
coordinator: 0.1630
person: 0.1586
request: 0.1511
fun: 0.1494
market: 0.1435


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Sample user profiles
sample_users = [
    {
        'skills': ["Digital Marketing", "Brand Management", "Social Media", "Graphic Design"],
        'experience': 5.5,
        'education': ["bachelor, Marketing University", "master, Digital Marketing Institute"],
        'location': "New York, USA",
        'remote_ok': True
    },
    {
        'skills': ["Python", "FastAPI", "SQL", "Machine Learning"],
        'experience': 3.0,
        'education': ["bachelor, Computer Science University"],
        'location': "San Francisco, USA",
        'remote_ok': False
    }
]

# Sample job postings
sample_jobs = [
    {
        'title': "Senior Marketing Manager",
        'description': "Looking for an experienced marketing professional to lead our digital marketing initiatives.",
        'required_skills': ["Digital Marketing", "Brand Management", "Team Leadership"],
        'required_experience': 5.0,
        'required_education': "bachelor",
        'location': "New York, USA",
        'remote_ok': True
    },
    {
        'title': "Python Developer",
        'description': "Seeking a Python developer with experience in web frameworks and machine learning.",
        'required_skills': ["Python", "FastAPI", "SQL"],
        'required_experience': 2.0,
        'required_education': "bachelor",
        'location': "San Francisco, USA",
        'remote_ok': False
    }
]

In [ ]:
# Prepare text for vectorization
def prepare_user_text(user):
    text = ' '.join(user['skills'])
    text += ' ' + ' '.join([edu.split(',')[0] for edu in user['education']])  # Add education levels
    return text

def prepare_job_text(job):
    text = job['title'] + ' ' + job['description']
    text += ' ' + ' '.join(job['required_skills'])
    text += ' ' + job['required_education']
    return text

# Preprocess and vectorize
user_texts = [preprocess_text(prepare_user_text(u)) for u in sample_users]
job_texts = [preprocess_text(prepare_job_text(j)) for j in sample_jobs]

# Transform texts to TFIDF vectors
user_vectors = tfidf.transform(user_texts)
job_vectors = tfidf.transform(job_texts)

# Calculate similarity
similarities = cosine_similarity(user_vectors, job_vectors)

# Print results
for i, user in enumerate(sample_users):
    print(f"\nUser {i+1}")
    print(f"Skills: {user['skills']}")
    print(f"Experience: {user['experience']} years")
    print(f"Education: {user['education']}")
    print(f"Location: {user['location']}")
    print("Top job matches:")
    
    user_similarities = similarities[i]
    top_matches = sorted(enumerate(user_similarities), key=lambda x: x[1], reverse=True)
    
    for job_idx, score in top_matches:
        job = sample_jobs[job_idx]
        print(f"\n- {job['title']}")
        print(f"  Similarity Score: {score:.4f}")
        print(f"  Required Experience: {job['required_experience']} years")
        print(f"  Location: {job['location']}")
        print(f"  Remote: {job['remote_ok']}")


User 1
Skills: ['Digital Marketing', 'Brand Management', 'Social Media', 'Graphic Design']
Experience: 5.5 years
Education: ['bachelor, Marketing University', 'master, Digital Marketing Institute']
Location: New York, USA
Top job matches:

- Senior Marketing Manager
  Similarity Score: 0.5632
  Required Experience: 5.0 years
  Location: New York, USA
  Remote: True

- Python Developer
  Similarity Score: 0.0556
  Required Experience: 2.0 years
  Location: San Francisco, USA
  Remote: False

User 2
Skills: ['Python', 'FastAPI', 'SQL', 'Machine Learning']
Experience: 3.0 years
Education: ['bachelor, Computer Science University']
Location: San Francisco, USA
Top job matches:

- Python Developer
  Similarity Score: 0.4662
  Required Experience: 2.0 years
  Location: San Francisco, USA
  Remote: False

- Senior Marketing Manager
  Similarity Score: 0.0738
  Required Experience: 5.0 years
  Location: New York, USA
  Remote: True


In [ ]:
def calculate_match_score(user, job, similarity_score):
    # Base score from TFIDF similarity
    total_score = similarity_score * 0.6  # 60% weight for content similarity
    
    # Experience match (20% weight)
    exp_score = 0.0
    if user['experience'] >= job['required_experience']:
        exp_score = 0.2
    elif user['experience'] >= job['required_experience'] * 0.8:  # Allow 80% of required
        exp_score = 0.1
    total_score += exp_score
    
    # Location match (10% weight)
    location_score = 0.1 if (user['location'] == job['location'] or 
                           (job['remote_ok'] and user['remote_ok'])) else 0.0
    total_score += location_score
    
    # Education match (10% weight)
    education_levels = {'high school': 1, 'bachelor': 2, 'master': 3, 'phd': 4}
    user_highest_edu = max([education_levels.get(edu.split(',')[0].strip().lower(), 0) 
                          for edu in user['education']])
    required_edu = education_levels.get(job['required_education'].lower(), 0)
    education_score = 0.1 if user_highest_edu >= required_edu else 0.0
    total_score += education_score
    
    return total_score

# Print results with combined scoring
for i, user in enumerate(sample_users):
    print(f"\nUser {i+1}")
    print(f"Skills: {user['skills']}")
    print(f"Experience: {user['experience']} years")
    print(f"Education: {user['education']}")
    print(f"Location: {user['location']}")
    print("Top job matches:")
    
    # Calculate combined scores
    user_similarities = similarities[i]
    combined_scores = [(idx, calculate_match_score(user, job, sim_score)) 
                      for idx, (job, sim_score) in enumerate(zip(sample_jobs, user_similarities))]
    
    # Sort by combined score
    top_matches = sorted(combined_scores, key=lambda x: x[1], reverse=True)
    
    for job_idx, score in top_matches:
        job = sample_jobs[job_idx]
        print(f"\n- {job['title']}")
        print(f"  Overall Match Score: {score:.4f}")
        print(f"  TFIDF Similarity: {user_similarities[job_idx]:.4f}")
        print(f"  Required Experience: {job['required_experience']} years")
        print(f"  Required Education: {job['required_education']}")
        print(f"  Location: {job['location']}")
        print(f"  Remote: {job['remote_ok']}")


User 1
Skills: ['Digital Marketing', 'Brand Management', 'Social Media', 'Graphic Design']
Experience: 5.5 years
Education: ['bachelor, Marketing University', 'master, Digital Marketing Institute']
Location: New York, USA
Top job matches:

- Senior Marketing Manager
  Overall Match Score: 0.7379
  TFIDF Similarity: 0.5632
  Required Experience: 5.0 years
  Required Education: bachelor
  Location: New York, USA
  Remote: True

- Python Developer
  Overall Match Score: 0.3334
  TFIDF Similarity: 0.0556
  Required Experience: 2.0 years
  Required Education: bachelor
  Location: San Francisco, USA
  Remote: False

User 2
Skills: ['Python', 'FastAPI', 'SQL', 'Machine Learning']
Experience: 3.0 years
Education: ['bachelor, Computer Science University']
Location: San Francisco, USA
Top job matches:

- Python Developer
  Overall Match Score: 0.6797
  TFIDF Similarity: 0.4662
  Required Experience: 2.0 years
  Required Education: bachelor
  Location: San Francisco, USA
  Remote: False

- Senio

In [2]:
df = pd.read_csv('linkedin_jobs.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 31 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   job_id                      123849 non-null  int64  
 1   company_name                122130 non-null  object 
 2   title                       123849 non-null  object 
 3   description                 123842 non-null  object 
 4   max_salary                  29793 non-null   float64
 5   pay_period                  36073 non-null   object 
 6   location                    123849 non-null  object 
 7   company_id                  122132 non-null  float64
 8   views                       122160 non-null  float64
 9   med_salary                  6280 non-null    float64
 10  min_salary                  29793 non-null   float64
 11  formatted_work_type         123849 non-null  object 
 12  applies                     23320 non-null   float64
 13  original_liste

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('linkedin_jobs.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 31 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   job_id                      123849 non-null  int64  
 1   company_name                122130 non-null  object 
 2   title                       123849 non-null  object 
 3   description                 123842 non-null  object 
 4   max_salary                  29793 non-null   float64
 5   pay_period                  36073 non-null   object 
 6   location                    123849 non-null  object 
 7   company_id                  122132 non-null  float64
 8   views                       122160 non-null  float64
 9   med_salary                  6280 non-null    float64
 10  min_salary                  29793 non-null   float64
 11  formatted_work_type         123849 non-null  object 
 12  applies                     23320 non-null   float64
 13  original_liste

In [ ]:
tfidf_columns = ['title', 'description']
df_tfidf = df[tfidf_columns].copy()

In [ ]:
print("Dataset Info:")
df_tfidf.info()

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   title        123849 non-null  object
 1   description  123842 non-null  object
dtypes: object(2)
memory usage: 1.9+ MB


In [ ]:
print("\nNull values in each column:")
print(df_tfidf.isnull().sum())


Null values in each column:
title          0
description    7
dtype: int64


In [ ]:
df_clean = df_tfidf.dropna(subset=['description'])

In [ ]:
print("\nSample data:")
df_clean.head()


Sample data:


,title,description
0,Marketing Coordinator,Job descriptionA leading real estate firm in N...
1,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ..."
2,Assitant Restaurant Manager,The National Exemplar is accepting application...
3,Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...
4,Service Technician,Looking for HVAC service tech with experience ...


In [ ]:
df_clean['combined_text'] = df_clean['title'] + ' | ' + df_clean['description']

C:\Users\akstrov\AppData\Local\Temp\ipykernel_16128\62390856.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['combined_text'] = df_clean['title'] + ' | ' + df_clean['description']


In [ ]:
print("Sample combined text:")
df_clean['combined_text'].head()

Sample combined text:


0    Marketing Coordinator | Job descriptionA leadi...
1    Mental Health Therapist/Counselor | At Aspen T...
2    Assitant Restaurant Manager | The National Exe...
3    Senior Elder Law / Trusts and Estates Associat...
4     Service Technician | Looking for HVAC service...
Name: combined_text, dtype: object

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

def preprocess_text(text):
    try:
        # Convert to lowercase
        text = str(text).lower()
        
        # Remove special characters and numbers
        text = re.sub(r'[^a-zA-Z\s]', ' ', text)
        
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text).strip()
        
        # Tokenize
        tokens = word_tokenize(text)
        
        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]
        
        # POS tagging
        pos_tags = nltk.pos_tag(tokens)
        
        # Lemmatize with POS
        lemmatizer = WordNetLemmatizer()
        lemmatized_tokens = []
        for token, tag in pos_tags:
            if tag.startswith('VB'):  # Verb
                lem_token = lemmatizer.lemmatize(token, pos='v')
            elif tag.startswith('NN'):  # Noun
                lem_token = lemmatizer.lemmatize(token, pos='n')
            elif tag.startswith('JJ'):  # Adjective
                lem_token = lemmatizer.lemmatize(token, pos='a')
            else:
                lem_token = lemmatizer.lemmatize(token)
            lemmatized_tokens.append(lem_token)
        
        return ' '.join(lemmatized_tokens)
        
    except Exception as e:
        print(f"Error processing text: {e}")
        return ''

# Test with the same text
test_text = "Marketing Coordinator | Job description: coordinating marketing campaigns, analyzing metrics, working with teams, managing projects"
processed = preprocess_text(test_text)

print("\nTest lemmatization:")
print("Original:", test_text)
print("Processed:", processed)


Test lemmatization:
Original: Marketing Coordinator | Job description: coordinating marketing campaigns, analyzing metrics, working with teams, managing projects
Processed: marketing coordinator job description coordinate marketing campaign analyze metric work team manage project


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
df_clean['processed_text'] = df_clean['combined_text'].apply(preprocess_text)

C:\Users\akstrov\AppData\Local\Temp\ipykernel_16128\1890915169.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['processed_text'] = df_clean['combined_text'].apply(preprocess_text)


In [ ]:
print("Original vs Processed Text:")
for orig, proc in zip(df_clean['combined_text'].head(2), df_clean['processed_text'].head(2)):
    print("\nOriginal:", orig[:200], "...")
    print("Processed:", proc[:200], "...")

Original vs Processed Text:

Original: Marketing Coordinator | Job descriptionA leading real estate firm in New Jersey is seeking an administrative Marketing Coordinator with some experience in graphic design. You will be working closely w ...
Processed: marketing coordinator job descriptiona lead real estate firm new jersey seek administrative marketing coordinator experience graphic design work closely fun kind ambitious member sale team dynamic exe ...

Original: Mental Health Therapist/Counselor | At Aspen Therapy and Wellness , we are committed to serving clients with best practices to help them with change, improvements and better quality of life. We believ ...
Processed: mental health therapist counselor aspen therapy wellness commit serve client best practice help change improvement better quality life believe provide secure supportive environment grow clinician lear ...


In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Create TFIDF vectorizer
tfidf = TfidfVectorizer(
    max_features=1000,  # Limit features to most frequent 1000 terms
    min_df=5,          # Ignore terms that appear in less than 5 documents
    max_df=0.95,       # Ignore terms that appear in more than 95% of documents
    ngram_range=(1, 2) # Include both unigrams and bigrams
)

# Fit and transform the preprocessed text
tfidf_matrix = tfidf.fit_transform(df_clean['processed_text'])

# Get feature names (vocabulary)
feature_names = tfidf.get_feature_names_out()

# Print some statistics
print(f"Shape of TFIDF matrix: {tfidf_matrix.shape}")
print(f"\nNumber of features: {len(feature_names)}")
print("\nSample features (first 20):")
print(feature_names[:20])

# Show sample vector
print("\nSample document vector (first document):")
sample_vector = tfidf_matrix[0].toarray()[0]
sample_terms = [(term, score) for term, score in zip(feature_names, sample_vector) if score > 0]
print("Non-zero terms and their TFIDF scores:")
for term, score in sorted(sample_terms, key=lambda x: x[1], reverse=True)[:10]:
    print(f"{term}: {score:.4f}")

Shape of TFIDF matrix: (123842, 1000)

Number of features: 1000

Sample features (first 20):
['ability' 'ability work' 'able' 'accept' 'access' 'accommodation'
 'accord' 'accordance' 'account' 'accountability' 'accounting' 'accredit'
 'accuracy' 'accurate' 'accurately' 'achieve' 'across' 'act' 'action'
 'active']

Sample document vector (first document):
Non-zero terms and their TFIDF scores:
marketing: 0.3849
design: 0.2246
event: 0.1816
brand: 0.1785
organize: 0.1768
coordinator: 0.1630
person: 0.1586
request: 0.1511
fun: 0.1494
market: 0.1435


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Sample user profiles
sample_users = [
    {
        'skills': ["Digital Marketing", "Brand Management", "Social Media", "Graphic Design"],
        'experience': 5.5,
        'education': ["bachelor, Marketing University", "master, Digital Marketing Institute"],
        'location': "New York, USA",
        'remote_ok': True
    },
    {
        'skills': ["Python", "FastAPI", "SQL", "Machine Learning"],
        'experience': 3.0,
        'education': ["bachelor, Computer Science University"],
        'location': "San Francisco, USA",
        'remote_ok': False
    }
]

# Sample job postings
sample_jobs = [
    {
        'title': "Senior Marketing Manager",
        'description': "Looking for an experienced marketing professional to lead our digital marketing initiatives.",
        'required_skills': ["Digital Marketing", "Brand Management", "Team Leadership"],
        'required_experience': 5.0,
        'required_education': "bachelor",
        'location': "New York, USA",
        'remote_ok': True
    },
    {
        'title': "Python Developer",
        'description': "Seeking a Python developer with experience in web frameworks and machine learning.",
        'required_skills': ["Python", "FastAPI", "SQL"],
        'required_experience': 2.0,
        'required_education': "bachelor",
        'location': "San Francisco, USA",
        'remote_ok': False
    }
]

In [ ]:
# Prepare text for vectorization
def prepare_user_text(user):
    text = ' '.join(user['skills'])
    text += ' ' + ' '.join([edu.split(',')[0] for edu in user['education']])  # Add education levels
    return text

def prepare_job_text(job):
    text = job['title'] + ' ' + job['description']
    text += ' ' + ' '.join(job['required_skills'])
    text += ' ' + job['required_education']
    return text

# Preprocess and vectorize
user_texts = [preprocess_text(prepare_user_text(u)) for u in sample_users]
job_texts = [preprocess_text(prepare_job_text(j)) for j in sample_jobs]

# Transform texts to TFIDF vectors
user_vectors = tfidf.transform(user_texts)
job_vectors = tfidf.transform(job_texts)

# Calculate similarity
similarities = cosine_similarity(user_vectors, job_vectors)

# Print results
for i, user in enumerate(sample_users):
    print(f"\nUser {i+1}")
    print(f"Skills: {user['skills']}")
    print(f"Experience: {user['experience']} years")
    print(f"Education: {user['education']}")
    print(f"Location: {user['location']}")
    print("Top job matches:")
    
    user_similarities = similarities[i]
    top_matches = sorted(enumerate(user_similarities), key=lambda x: x[1], reverse=True)
    
    for job_idx, score in top_matches:
        job = sample_jobs[job_idx]
        print(f"\n- {job['title']}")
        print(f"  Similarity Score: {score:.4f}")
        print(f"  Required Experience: {job['required_experience']} years")
        print(f"  Location: {job['location']}")
        print(f"  Remote: {job['remote_ok']}")


User 1
Skills: ['Digital Marketing', 'Brand Management', 'Social Media', 'Graphic Design']
Experience: 5.5 years
Education: ['bachelor, Marketing University', 'master, Digital Marketing Institute']
Location: New York, USA
Top job matches:

- Senior Marketing Manager
  Similarity Score: 0.5632
  Required Experience: 5.0 years
  Location: New York, USA
  Remote: True

- Python Developer
  Similarity Score: 0.0556
  Required Experience: 2.0 years
  Location: San Francisco, USA
  Remote: False

User 2
Skills: ['Python', 'FastAPI', 'SQL', 'Machine Learning']
Experience: 3.0 years
Education: ['bachelor, Computer Science University']
Location: San Francisco, USA
Top job matches:

- Python Developer
  Similarity Score: 0.4662
  Required Experience: 2.0 years
  Location: San Francisco, USA
  Remote: False

- Senior Marketing Manager
  Similarity Score: 0.0738
  Required Experience: 5.0 years
  Location: New York, USA
  Remote: True


In [ ]:
def calculate_match_score(user, job, similarity_score):
    # Base score from TFIDF similarity
    total_score = similarity_score * 0.6  # 60% weight for content similarity
    
    # Experience match (20% weight)
    exp_score = 0.0
    if user['experience'] >= job['required_experience']:
        exp_score = 0.2
    elif user['experience'] >= job['required_experience'] * 0.8:  # Allow 80% of required
        exp_score = 0.1
    total_score += exp_score
    
    # Location match (10% weight)
    location_score = 0.1 if (user['location'] == job['location'] or 
                           (job['remote_ok'] and user['remote_ok'])) else 0.0
    total_score += location_score
    
    # Education match (10% weight)
    education_levels = {'high school': 1, 'bachelor': 2, 'master': 3, 'phd': 4}
    user_highest_edu = max([education_levels.get(edu.split(',')[0].strip().lower(), 0) 
                          for edu in user['education']])
    required_edu = education_levels.get(job['required_education'].lower(), 0)
    education_score = 0.1 if user_highest_edu >= required_edu else 0.0
    total_score += education_score
    
    return total_score

# Print results with combined scoring
for i, user in enumerate(sample_users):
    print(f"\nUser {i+1}")
    print(f"Skills: {user['skills']}")
    print(f"Experience: {user['experience']} years")
    print(f"Education: {user['education']}")
    print(f"Location: {user['location']}")
    print("Top job matches:")
    
    # Calculate combined scores
    user_similarities = similarities[i]
    combined_scores = [(idx, calculate_match_score(user, job, sim_score)) 
                      for idx, (job, sim_score) in enumerate(zip(sample_jobs, user_similarities))]
    
    # Sort by combined score
    top_matches = sorted(combined_scores, key=lambda x: x[1], reverse=True)
    
    for job_idx, score in top_matches:
        job = sample_jobs[job_idx]
        print(f"\n- {job['title']}")
        print(f"  Overall Match Score: {score:.4f}")
        print(f"  TFIDF Similarity: {user_similarities[job_idx]:.4f}")
        print(f"  Required Experience: {job['required_experience']} years")
        print(f"  Required Education: {job['required_education']}")
        print(f"  Location: {job['location']}")
        print(f"  Remote: {job['remote_ok']}")


User 1
Skills: ['Digital Marketing', 'Brand Management', 'Social Media', 'Graphic Design']
Experience: 5.5 years
Education: ['bachelor, Marketing University', 'master, Digital Marketing Institute']
Location: New York, USA
Top job matches:

- Senior Marketing Manager
  Overall Match Score: 0.7379
  TFIDF Similarity: 0.5632
  Required Experience: 5.0 years
  Required Education: bachelor
  Location: New York, USA
  Remote: True

- Python Developer
  Overall Match Score: 0.3334
  TFIDF Similarity: 0.0556
  Required Experience: 2.0 years
  Required Education: bachelor
  Location: San Francisco, USA
  Remote: False

User 2
Skills: ['Python', 'FastAPI', 'SQL', 'Machine Learning']
Experience: 3.0 years
Education: ['bachelor, Computer Science University']
Location: San Francisco, USA
Top job matches:

- Python Developer
  Overall Match Score: 0.6797
  TFIDF Similarity: 0.4662
  Required Experience: 2.0 years
  Required Education: bachelor
  Location: San Francisco, USA
  Remote: False

- Senio

In [5]:
tfidf_columns = ['title', 'description']
df_tfidf = df[tfidf_columns].copy()

In [6]:
print("Dataset Info:")
df_tfidf.info()

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   title        123849 non-null  object
 1   description  123842 non-null  object
dtypes: object(2)
memory usage: 1.9+ MB


In [7]:
print("\nNull values in each column:")
print(df_tfidf.isnull().sum())


Null values in each column:
title          0
description    7
dtype: int64


In [8]:
df_clean = df_tfidf.dropna(subset=['description'])

In [9]:
print("\nSample data:")
df_clean.head()


Sample data:


,title,description
0,Marketing Coordinator,Job descriptionA leading real estate firm in N...
1,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ..."
2,Assitant Restaurant Manager,The National Exemplar is accepting application...
3,Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...
4,Service Technician,Looking for HVAC service tech with experience ...


In [10]:
df_clean['combined_text'] = df_clean['title'] + ' | ' + df_clean['description']

C:\Users\akstrov\AppData\Local\Temp\ipykernel_16128\62390856.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['combined_text'] = df_clean['title'] + ' | ' + df_clean['description']


In [11]:
print("Sample combined text:")
df_clean['combined_text'].head()

Sample combined text:


0    Marketing Coordinator | Job descriptionA leadi...
1    Mental Health Therapist/Counselor | At Aspen T...
2    Assitant Restaurant Manager | The National Exe...
3    Senior Elder Law / Trusts and Estates Associat...
4     Service Technician | Looking for HVAC service...
Name: combined_text, dtype: object

In [12]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [13]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [22]:
import nltk
nltk.download('averaged_perceptron_tagger')

def preprocess_text(text):
    try:
        # Convert to lowercase
        text = str(text).lower()
        
        # Remove special characters and numbers
        text = re.sub(r'[^a-zA-Z\s]', ' ', text)
        
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text).strip()
        
        # Tokenize
        tokens = word_tokenize(text)
        
        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]
        
        # POS tagging
        pos_tags = nltk.pos_tag(tokens)
        
        # Lemmatize with POS
        lemmatizer = WordNetLemmatizer()
        lemmatized_tokens = []
        for token, tag in pos_tags:
            if tag.startswith('VB'):  # Verb
                lem_token = lemmatizer.lemmatize(token, pos='v')
            elif tag.startswith('NN'):  # Noun
                lem_token = lemmatizer.lemmatize(token, pos='n')
            elif tag.startswith('JJ'):  # Adjective
                lem_token = lemmatizer.lemmatize(token, pos='a')
            else:
                lem_token = lemmatizer.lemmatize(token)
            lemmatized_tokens.append(lem_token)
        
        return ' '.join(lemmatized_tokens)
        
    except Exception as e:
        print(f"Error processing text: {e}")
        return ''

# Test with the same text
test_text = "Marketing Coordinator | Job description: coordinating marketing campaigns, analyzing metrics, working with teams, managing projects"
processed = preprocess_text(test_text)

print("\nTest lemmatization:")
print("Original:", test_text)
print("Processed:", processed)


Test lemmatization:
Original: Marketing Coordinator | Job description: coordinating marketing campaigns, analyzing metrics, working with teams, managing projects
Processed: marketing coordinator job description coordinate marketing campaign analyze metric work team manage project


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\akstrov\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [23]:
df_clean['processed_text'] = df_clean['combined_text'].apply(preprocess_text)

C:\Users\akstrov\AppData\Local\Temp\ipykernel_16128\1890915169.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['processed_text'] = df_clean['combined_text'].apply(preprocess_text)


In [25]:
print("Original vs Processed Text:")
for orig, proc in zip(df_clean['combined_text'].head(2), df_clean['processed_text'].head(2)):
    print("\nOriginal:", orig[:200], "...")
    print("Processed:", proc[:200], "...")

Original vs Processed Text:

Original: Marketing Coordinator | Job descriptionA leading real estate firm in New Jersey is seeking an administrative Marketing Coordinator with some experience in graphic design. You will be working closely w ...
Processed: marketing coordinator job descriptiona lead real estate firm new jersey seek administrative marketing coordinator experience graphic design work closely fun kind ambitious member sale team dynamic exe ...

Original: Mental Health Therapist/Counselor | At Aspen Therapy and Wellness , we are committed to serving clients with best practices to help them with change, improvements and better quality of life. We believ ...
Processed: mental health therapist counselor aspen therapy wellness commit serve client best practice help change improvement better quality life believe provide secure supportive environment grow clinician lear ...


In [26]:

from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
# Create TFIDF vectorizer
tfidf = TfidfVectorizer(
    max_features=1000,  # Limit features to most frequent 1000 terms
    min_df=5,          # Ignore terms that appear in less than 5 documents
    max_df=0.95,       # Ignore terms that appear in more than 95% of documents
    ngram_range=(1, 2) # Include both unigrams and bigrams
)

# Fit and transform the preprocessed text
tfidf_matrix = tfidf.fit_transform(df_clean['processed_text'])

# Get feature names (vocabulary)
feature_names = tfidf.get_feature_names_out()

# Print some statistics
print(f"Shape of TFIDF matrix: {tfidf_matrix.shape}")
print(f"\nNumber of features: {len(feature_names)}")
print("\nSample features (first 20):")
print(feature_names[:20])

# Show sample vector
print("\nSample document vector (first document):")
sample_vector = tfidf_matrix[0].toarray()[0]
sample_terms = [(term, score) for term, score in zip(feature_names, sample_vector) if score > 0]
print("Non-zero terms and their TFIDF scores:")
for term, score in sorted(sample_terms, key=lambda x: x[1], reverse=True)[:10]:
    print(f"{term}: {score:.4f}")

Shape of TFIDF matrix: (123842, 1000)

Number of features: 1000

Sample features (first 20):
['ability' 'ability work' 'able' 'accept' 'access' 'accommodation'
 'accord' 'accordance' 'account' 'accountability' 'accounting' 'accredit'
 'accuracy' 'accurate' 'accurately' 'achieve' 'across' 'act' 'action'
 'active']

Sample document vector (first document):
Non-zero terms and their TFIDF scores:
marketing: 0.3849
design: 0.2246
event: 0.1816
brand: 0.1785
organize: 0.1768
coordinator: 0.1630
person: 0.1586
request: 0.1511
fun: 0.1494
market: 0.1435


In [28]:
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
# Sample user profiles
sample_users = [
    {
        'skills': ["Digital Marketing", "Brand Management", "Social Media", "Graphic Design"],
        'experience': 5.5,
        'education': ["bachelor, Marketing University", "master, Digital Marketing Institute"],
        'location': "New York, USA",
        'remote_ok': True
    },
    {
        'skills': ["Python", "FastAPI", "SQL", "Machine Learning"],
        'experience': 3.0,
        'education': ["bachelor, Computer Science University"],
        'location': "San Francisco, USA",
        'remote_ok': False
    }
]

# Sample job postings
sample_jobs = [
    {
        'title': "Senior Marketing Manager",
        'description': "Looking for an experienced marketing professional to lead our digital marketing initiatives.",
        'required_skills': ["Digital Marketing", "Brand Management", "Team Leadership"],
        'required_experience': 5.0,
        'required_education': "bachelor",
        'location': "New York, USA",
        'remote_ok': True
    },
    {
        'title': "Python Developer",
        'description': "Seeking a Python developer with experience in web frameworks and machine learning.",
        'required_skills': ["Python", "FastAPI", "SQL"],
        'required_experience': 2.0,
        'required_education': "bachelor",
        'location': "San Francisco, USA",
        'remote_ok': False
    }
]

In [30]:
# Prepare text for vectorization
def prepare_user_text(user):
    text = ' '.join(user['skills'])
    text += ' ' + ' '.join([edu.split(',')[0] for edu in user['education']])  # Add education levels
    return text

def prepare_job_text(job):
    text = job['title'] + ' ' + job['description']
    text += ' ' + ' '.join(job['required_skills'])
    text += ' ' + job['required_education']
    return text

# Preprocess and vectorize
user_texts = [preprocess_text(prepare_user_text(u)) for u in sample_users]
job_texts = [preprocess_text(prepare_job_text(j)) for j in sample_jobs]

# Transform texts to TFIDF vectors
user_vectors = tfidf.transform(user_texts)
job_vectors = tfidf.transform(job_texts)

# Calculate similarity
similarities = cosine_similarity(user_vectors, job_vectors)

# Print results
for i, user in enumerate(sample_users):
    print(f"\nUser {i+1}")
    print(f"Skills: {user['skills']}")
    print(f"Experience: {user['experience']} years")
    print(f"Education: {user['education']}")
    print(f"Location: {user['location']}")
    print("Top job matches:")
    
    user_similarities = similarities[i]
    top_matches = sorted(enumerate(user_similarities), key=lambda x: x[1], reverse=True)
    
    for job_idx, score in top_matches:
        job = sample_jobs[job_idx]
        print(f"\n- {job['title']}")
        print(f"  Similarity Score: {score:.4f}")
        print(f"  Required Experience: {job['required_experience']} years")
        print(f"  Location: {job['location']}")
        print(f"  Remote: {job['remote_ok']}")


User 1
Skills: ['Digital Marketing', 'Brand Management', 'Social Media', 'Graphic Design']
Experience: 5.5 years
Education: ['bachelor, Marketing University', 'master, Digital Marketing Institute']
Location: New York, USA
Top job matches:

- Senior Marketing Manager
  Similarity Score: 0.5632
  Required Experience: 5.0 years
  Location: New York, USA
  Remote: True

- Python Developer
  Similarity Score: 0.0556
  Required Experience: 2.0 years
  Location: San Francisco, USA
  Remote: False

User 2
Skills: ['Python', 'FastAPI', 'SQL', 'Machine Learning']
Experience: 3.0 years
Education: ['bachelor, Computer Science University']
Location: San Francisco, USA
Top job matches:

- Python Developer
  Similarity Score: 0.4662
  Required Experience: 2.0 years
  Location: San Francisco, USA
  Remote: False

- Senior Marketing Manager
  Similarity Score: 0.0738
  Required Experience: 5.0 years
  Location: New York, USA
  Remote: True


In [31]:
def calculate_match_score(user, job, similarity_score):
    # Base score from TFIDF similarity
    total_score = similarity_score * 0.6  # 60% weight for content similarity
    
    # Experience match (20% weight)
    exp_score = 0.0
    if user['experience'] >= job['required_experience']:
        exp_score = 0.2
    elif user['experience'] >= job['required_experience'] * 0.8:  # Allow 80% of required
        exp_score = 0.1
    total_score += exp_score
    
    # Location match (10% weight)
    location_score = 0.1 if (user['location'] == job['location'] or 
                           (job['remote_ok'] and user['remote_ok'])) else 0.0
    total_score += location_score
    
    # Education match (10% weight)
    education_levels = {'high school': 1, 'bachelor': 2, 'master': 3, 'phd': 4}
    user_highest_edu = max([education_levels.get(edu.split(',')[0].strip().lower(), 0) 
                          for edu in user['education']])
    required_edu = education_levels.get(job['required_education'].lower(), 0)
    education_score = 0.1 if user_highest_edu >= required_edu else 0.0
    total_score += education_score
    
    return total_score

# Print results with combined scoring
for i, user in enumerate(sample_users):
    print(f"\nUser {i+1}")
    print(f"Skills: {user['skills']}")
    print(f"Experience: {user['experience']} years")
    print(f"Education: {user['education']}")
    print(f"Location: {user['location']}")
    print("Top job matches:")
    
    # Calculate combined scores
    user_similarities = similarities[i]
    combined_scores = [(idx, calculate_match_score(user, job, sim_score)) 
                      for idx, (job, sim_score) in enumerate(zip(sample_jobs, user_similarities))]
    
    # Sort by combined score
    top_matches = sorted(combined_scores, key=lambda x: x[1], reverse=True)
    
    for job_idx, score in top_matches:
        job = sample_jobs[job_idx]
        print(f"\n- {job['title']}")
        print(f"  Overall Match Score: {score:.4f}")
        print(f"  TFIDF Similarity: {user_similarities[job_idx]:.4f}")
        print(f"  Required Experience: {job['required_experience']} years")
        print(f"  Required Education: {job['required_education']}")
        print(f"  Location: {job['location']}")
        print(f"  Remote: {job['remote_ok']}")


User 1
Skills: ['Digital Marketing', 'Brand Management', 'Social Media', 'Graphic Design']
Experience: 5.5 years
Education: ['bachelor, Marketing University', 'master, Digital Marketing Institute']
Location: New York, USA
Top job matches:

- Senior Marketing Manager
  Overall Match Score: 0.7379
  TFIDF Similarity: 0.5632
  Required Experience: 5.0 years
  Required Education: bachelor
  Location: New York, USA
  Remote: True

- Python Developer
  Overall Match Score: 0.3334
  TFIDF Similarity: 0.0556
  Required Experience: 2.0 years
  Required Education: bachelor
  Location: San Francisco, USA
  Remote: False

User 2
Skills: ['Python', 'FastAPI', 'SQL', 'Machine Learning']
Experience: 3.0 years
Education: ['bachelor, Computer Science University']
Location: San Francisco, USA
Top job matches:

- Python Developer
  Overall Match Score: 0.6797
  TFIDF Similarity: 0.4662
  Required Experience: 2.0 years
  Required Education: bachelor
  Location: San Francisco, USA
  Remote: False

- Senio